In [2]:
import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup

import json
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import time

**DOC**
- https://ramiro.org/notebook/us-presidents-causes-of-death/
- https://janakiev.com/blog/wikidata-mayors/
- https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial
- https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples
- https://stackoverflow.com/questions/37119412/wikidata-sparql-how-to-access-award-point-of-time-and-work-from-a-people
- https://www.wikidata.org/wiki/Wikidata:Request_a_query/Archive/2017/10

### Très bonne vidéo expliquant comment construire une requête sur Wikidata
- https://www.youtube.com/watch?v=b3ft3CzkLYk&t=2409s

**Pour tester sa requète Wikidata :**
- https://query.wikidata.org/

#### Une fois la requête construite, on peut la lancer

### On va chercher ici tous les prix Nobel de Physique avec date de naissance et genre
- La référence du Nobel de Physique : Q38104

In [12]:
endpoint_url = "https://query.wikidata.org/sparql"

query = """PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?item ?itemLabel ?date ?birth ?genderLabel ?awardLabel ?countryLabel
{
    ?item wdt:P166 wd:Q38104 .
  OPTIONAL {
  ?item p:P166 ?awardstatement .
  ?awardstatement ps:P166 ?award .
  ?awardstatement pq:P585 ?date .
  ?item wdt:P569 ?birth.
#   ?item wdt:P570 ?death .
  ?item wdt:P21 ?gender.
  ?item wdt:P27 ?country.
    }
     FILTER(?award = wd:Q38104).
    SERVICE wikibase:label {               # ... include the labels
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr"
    }
}
ORDER BY DESC(?date)"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


nobel_physique_dict = get_results(endpoint_url, query)

### On obtient un dictionnaire, dont on cherche les premières clés pour convertir en format tabulaire

In [9]:
type(nobel_physique_dict)

dict

In [10]:
nobel_physique_dict.keys()

dict_keys(['head', 'results'])

In [11]:
nobel_physique_dict['results'].keys()

dict_keys(['bindings'])

### On convertit en DataFrame

In [13]:
df_nobels_physics = pd.json_normalize(nobel_physique_dict['results']['bindings'])

In [40]:
df_nobels_physics

,item.type,item.value,birth.datatype,birth.type,birth.value,date.datatype,date.type,date.value,itemLabel.xml:lang,itemLabel.type,itemLabel.value,genderLabel.xml:lang,genderLabel.type,genderLabel.value,awardLabel.xml:lang,awardLabel.type,awardLabel.value,countryLabel.xml:lang,countryLabel.type,countryLabel.value
0,uri,http://www.wikidata.org/entity/Q109370,http://www.w3.org/2001/XMLSchema#dateTime,literal,1931-10-25T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,2021-01-01T00:00:00Z,fr,literal,Klaus Hasselmann,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,Allemagne
1,uri,http://www.wikidata.org/entity/Q1235614,http://www.w3.org/2001/XMLSchema#dateTime,literal,1948-08-04T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,2021-01-01T00:00:00Z,fr,literal,Giorgio Parisi,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,Italie
2,uri,http://www.wikidata.org/entity/Q3675789,http://www.w3.org/2001/XMLSchema#dateTime,literal,1931-09-21T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,2021-01-01T00:00:00Z,fr,literal,Syukuro Manabe,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,Japon
3,uri,http://www.wikidata.org/entity/Q3675789,http://www.w3.org/2001/XMLSchema#dateTime,literal,1931-09-21T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,2021-01-01T00:00:00Z,fr,literal,Syukuro Manabe,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,États-Unis
4,uri,http://www.wikidata.org/entity/Q65807,http://www.w3.org/2001/XMLSchema#dateTime,literal,1952-03-24T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,2020-01-01T00:00:00Z,fr,literal,Reinhard Genzel,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,Allemagne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,uri,http://www.wikidata.org/entity/Q37463,http://www.w3.org/2001/XMLSchema#dateTime,literal,1859-05-15T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1903-01-01T00:00:00Z,fr,literal,Pierre Curie,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,France
311,uri,http://www.wikidata.org/entity/Q41269,http://www.w3.org/2001/XMLSchema#dateTime,literal,1852-12-15T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1903-01-01T00:00:00Z,fr,literal,Henri Becquerel,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,France
312,uri,http://www.wikidata.org/entity/Q79000,http://www.w3.org/2001/XMLSchema#dateTime,literal,1865-05-25T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1902-01-01T00:00:00Z,fr,literal,Pieter Zeeman,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,Royaume des Pays-Bas
313,uri,http://www.wikidata.org/entity/Q41688,http://www.w3.org/2001/XMLSchema#dateTime,literal,1853-07-18T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1902-01-01T00:00:00Z,fr,literal,Hendrik Antoon Lorentz,fr,literal,masculin,fr,literal,prix Nobel de physique,fr,literal,Royaume des Pays-Bas


#### A présent, on cherche tous les prix Nobels de toutes les disciplines

#### On crée un dictionnaire avec les différents prix, et on met la requête dans une fonction

In [16]:
dict_nobels = {'chimie': 'Q44585',
 'physique': 'Q38104',
 'Litterature' :  'Q37922',
'paix' : 'Q35637',
'medecine' : 'Q80061',
'economie' : 'Q47170'}

In [17]:
### La fonction rechechera en fonction de la référence du code demandé, et retournera une DataFrame
def get_nobel(nobel_type):
    query = """PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    
    SELECT ?item ?itemLabel ?date ?birth ?genderLabel ?awardLabel ?countryLabel
    {
        ?item wdt:P166 wd:""" + nobel_type + " ." + """
      OPTIONAL {
      ?item p:P166 ?awardstatement .
      ?awardstatement ps:P166 ?award .
      ?awardstatement pq:P585 ?date .
      ?item wdt:P569 ?birth.
    #   ?item wdt:P570 ?death .
      ?item wdt:P21 ?gender.
      ?item wdt:P27 ?country.
        }
         FILTER(?award = wd:""" + nobel_type + ")." + """
        SERVICE wikibase:label {               # ... include the labels
            bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr"
        }
    }
    ORDER BY DESC(?date)"""
    this_result = get_results(endpoint_url, query)
    this_df = pd.json_normalize(this_result['results']['bindings'])
    return this_df

In [18]:
### Boucle pour aller chercher tous les prix Nobel et les sauvegarder dans une liste de DataFrame qu'on assemblera ultérieurement
all_dfs = []
all_dfs_dicts_list = []
for e in dict_nobels.items():
    this_df = get_nobel(e[1])
    all_dfs.append(this_df)
    all_dfs_dicts_list.append({'nobel_type': e[0], 'df': this_df})
    print(e[0])
    time.sleep(2)

chimie
physique
Litterature
paix
medecine
economie


In [20]:
def find_allcountries(row, df):
    mask = ((df['birth.value'] == row['birth.value']) & (df['date.value'] == row['date.value']) &
           (df['itemLabel.value'] == row['itemLabel.value']) & (df['genderLabel.value'] == row['genderLabel.value']) & 
           (df['awardLabel.value'] == row['awardLabel.value']))
    return ', '.join(df[mask]['countryLabel.value'].values)

In [21]:
def calculate_age(born, thisDate):
    return thisDate.year - born.year - ((thisDate.month, thisDate.day) < (born.month, born.day))

In [401]:
all_nobels_df_f = all_nobels_df_f.sort_values('date')

In [26]:
# Un lauréat Nobel peut être de plusieurs pays, on les assemble dans une colone
for df in all_dfs:
    df['all_countries'] = df.apply(find_allcountries, args=(df,), axis=1)

#On concatène la liste de DataFrame dans une seule DF
all_dfs_concat =  pd.concat(all_dfs)

selected_cols = ['birth.value', 'date.value',
       'itemLabel.value', 'genderLabel.value','awardLabel.value','all_countries']

#On sélectionne certaines colonnes
all_nobels_df =  all_dfs_concat[selected_cols]

#On renomme les colonnes
all_nobels_df = all_nobels_df.rename(columns={'birth.value': 'birth', 
                                              'date.value' : 'date','genderLabel.value':'genre',
                                              'itemLabel.value' : 'name', 'awardLabel.value' : 'award'})

# Format des dates et calcul de l'âge
all_nobels_df['birth'] = pd.to_datetime(all_nobels_df['birth'].map(lambda x: x.split('T')[0][:5] + '12-10'))
all_nobels_df['date'] = pd.to_datetime(all_nobels_df['date'].map(lambda x: x.split('T')[0][:5] + '12-10'))
all_nobels_df['age'] = all_nobels_df.apply(lambda row: calculate_age(row['birth'], row['date']) , axis=1)

#Tri par date
all_nobels_df = all_nobels_df.sort_values(['date', 'award'])

#Il y avait des doublons, on dédoublone
all_nobels_df = all_nobels_df.drop_duplicates()

In [27]:
all_nobels_df

,birth,date,name,genre,award,all_countries,age
264,1852-12-10,1901-12-10,Jacobus Henricus Van 't Hoff,masculin,prix Nobel de chimie,Royaume des Pays-Bas,49
134,1828-12-10,1901-12-10,Henry Dunant,masculin,prix Nobel de la paix,"Suisse, France",73
136,1822-12-10,1901-12-10,Frédéric Passy,masculin,prix Nobel de la paix,France,79
181,1839-12-10,1901-12-10,Sully Prudhomme,masculin,prix Nobel de littérature,France,62
305,1854-12-10,1901-12-10,Emil Adolf von Behring,masculin,prix Nobel de physiologie ou médecine,"royaume de Prusse, Empire allemand",47
...,...,...,...,...,...,...,...
2,1948-12-10,2021-12-10,Giorgio Parisi,masculin,prix Nobel de physique,Italie,73
3,1931-12-10,2021-12-10,Klaus Hasselmann,masculin,prix Nobel de physique,Allemagne,90
0,1960-12-10,2021-12-10,Joshua Angrist,masculin,prix de la Banque de Suède en sciences économi...,"États-Unis, Israël",61
2,1956-12-10,2021-12-10,David Card,masculin,prix de la Banque de Suède en sciences économi...,Canada,65


In [ ]:
# Sauvegarde en csv
all_nobels_df.to_csv('data/data_nobels.csv', index=False)